In [1]:
from flask import Flask
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
import matplotlib.pyplot as plt

app = Flask(__name__)

@app.route("/")
def hello_world():
    return "Hello world"

@app.route("/download_stock")
def download_stock():
    stock_symbol = 'AAPL' 
    stock_data = yf.download(stock_symbol, start='2022-01-01', end='2022-12-31')
    return stock_data.to_csv("final.csv")

@app.route("/show_chart")    
def get_stock():
    df = pd.read_csv('final.csv')
    df.title.text = 'Stock'
    df["Status"] =[inc_dec(c,o) for c, o in zip(df.Close,df.Open)]
    df["Middle"] = (df.Open+df.Close)/2
    df["Height"] = abs(df.Close-df.Open)
    fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

    fig.show()

@app.route("/chart_bollinger")    
def chart_bollinger():
    df = pd.read_csv('final.csv')
    df = bollinger_bands(df, 20, 2)
    stock_symbol = 'AAPL' 


    # plot price
    # plt.figure(figsize=(15,5))
    # plt.plot(df['Date'], df['Adj Close'])
    # plt.title('Price chart (Adj Close) ' + str(stock_symbol))
    # plt.show()

    # fig = go.Figure(data=[
    #     go.Bar(name='Adj Close', x=df['Date'], y=df['Adj Close']),
    # ])

    # fig.show()

    # plot correspondingRSI values and significant levels
    plt.figure(figsize=(15,5))
    plt.title('Bollinger Bands chart ' + str(stock_symbol))
    plt.plot(df['Date'], df['Adj Close'])
    plt.plot(df['Date'], df['BU'], alpha=0.3)
    plt.plot(df['Date'], df['BL'], alpha=0.3)
    plt.plot(df['Date'], df['B_MA'], alpha=0.3)
    plt.fill_between(df['Date'], df['BU'], df['BL'], color='grey', alpha=0.1)

    plt.show()

def inc_dec(c, o):  #param ( c: Close , o: Open )
    if c > o:
        value="Increase"
    elif c < o:
        value="Decrease"
    else:
        value="Equal"
    return value

def bollinger_bands(df, n, m):
    # n = smoothing length
    # m = number of standard deviations away from MA
    
    #typical price
    TP = (df['High'] + df['Low'] + df['Close']) / 3
    # but we will use Adj close instead for now, depends
    data = TP
    #data = df['Adj Close']

    # takes one column from dataframe
    B_MA = pd.Series((data.rolling(n, min_periods=n).mean()), name='B_MA')
    sigma = data.rolling(n, min_periods=n).std() 
    
    BU = pd.Series((B_MA + m * sigma), name='BU')
    BL = pd.Series((B_MA - m * sigma), name='BL')
    
    df = df.join(B_MA)
    df = df.join(BU)
    df = df.join(BL)

    return df

if __name__ == "__main__":
    app.run()



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [08/Nov/2023 08:56:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2023 08:56:39] "GET / HTTP/1.1" 200 -
[2023-11-08 08:56:53,055] ERROR in app: Exception on /show_chart [GET]
Traceback (most recent call last):
  File "d:\Study\PythonEngineers\.venv\Lib\site-packages\flask\app.py", line 1455, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Study\PythonEngineers\.venv\Lib\site-packages\flask\app.py", line 869, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Study\PythonEngineers\.venv\Lib\site-packages\flask\app.py", line 867, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Study\PythonEngineers\.venv\Lib\site-packages\flask\app.py", line 852, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_arg